From: https://github.com/richzhang/PerceptualSimilarity

In [1]:
import os, sys
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
from skimage.measure import structural_similarity as ssim
import cv2
#from skimage.transform import rotate
from PIL import Image
import imagehash
#import argparse
#import shelve
import glob

import torch
from PerceptualSimilarity.util import util
from PerceptualSimilarity.models import dist_model as dm
from IPython import embed

In [2]:
use_gpu = True         # Whether to use GPU
spatial = False         # Return a spatial map of perceptual distance.
                       # Optional args spatial_shape and spatial_order control output shape and resampling filter: see DistModel.initialize() for details.

In [3]:
## Initializing the model
model = dm.DistModel()

In [4]:
# Linearly calibrated models
#model.initialize(model='net-lin',net='squeeze',use_gpu=use_gpu,spatial=spatial)
model.initialize(model='net-lin',net='alex',use_gpu=use_gpu,spatial=spatial)
#model.initialize(model='net-lin',net='vgg',use_gpu=use_gpu,spatial=spatial)

In [5]:
# Low-level metrics
# model.initialize(model='l2',colorspace='Lab')
# model.initialize(model='ssim',colorspace='RGB')
print('Model [%s] initialized'%model.name())

Model [net-lin [alex]] initialized


In [6]:
pdf = os.path.join(os.getcwd(), 'pdf')
CELLPHONE_IMG_PATH = os.path.join(pdf, 'minidataset_cell')
DB_PATH            = os.path.join(pdf, 'mini_dataset_png')

In [7]:
png_paths = glob.glob(os.path.join(DB_PATH, '*.png'))

In [65]:
db = {}

In [66]:
testImgPath = os.path.join(CELLPHONE_IMG_PATH, 'IMG_9150.jpg') #this is where you choose the query image

In [67]:
ex_ref = util.im2tensor(util.load_image(testImgPath))

In [68]:
ex_ref.shape

torch.Size([1, 3, 4032, 3024])

In [69]:
for i in range(len(png_paths)):
    imagePath = png_paths[i]
    image = util.im2tensor(util.load_image(imagePath))
    
    image = image.resize_(ex_ref.shape)
    
    #print(image.shape)
    
    distance = model.forward(ex_ref, image)
    
    filename = imagePath[imagePath.rfind("/") + 1:]
    
    db[filename] = distance

In [70]:
db.keys() #should give filenames of what showed up in the database

['305764_page_88.png',
 '53399_page_2.png',
 '51212_page_385.png',
 '57895_page_2.png',
 '02235_page_77.png',
 '490343_page_9.png',
 '320363_page_291.png',
 '278240_page_201.png',
 '433671_page_7.png',
 '293281_page_253.png',
 '03031_page_19.png',
 '24577_page_142.png',
 '378078_page_75.png',
 '517401_page_154.png',
 '24547_page_66.png',
 '276011_page_23.png',
 '106106_page_41.png',
 '114900_page_4.png',
 '482211_page_1.png',
 '88879_page_355.png',
 '241869_page_42.png',
 '279776_page_18.png',
 '27113_page_16.png',
 '15562_page_57.png',
 '340340_page_96.png',
 '333433_page_30.png',
 '01685_page_24.png',
 '16314_page_29.png',
 '25315_page_2.png',
 '314961_page_4.png',
 '512324_page_30.png',
 '02331_page_69.png',
 '65711_page_23.png',
 '03366_page_36.png',
 '253851_page_2.png',
 '166951_page_33.png',
 '517025_page_594.png',
 '27475_page_7.png',
 '113413_page_1.png',
 '295562_page_85.png',
 '50754_page_1.png',
 '98290_page_46.png',
 '440770_page_289.png',
 '172491_page_2.png',
 '247445_pa

In [71]:
db['209693_page_3.png'][0] #see what ground truth's score is

0.35600573

In [72]:
scoreList = []

In [73]:
for key in db.keys():
    val = db[key]
    score = val[0]
    scoreList.append([key,score])

In [74]:
scoreList = sorted(scoreList,key=lambda l:l[1], reverse=True)

In [75]:
print(scoreList[0:60])

[['06028_page_50.png', 0.52358383], ['88639_page_6.png', 0.51095271], ['02267_page_2.png', 0.50848877], ['01380_page_5.png', 0.50188613], ['20018_page_3.png', 0.49163902], ['57857_page_22.png', 0.48587799], ['19459_page_3.png', 0.47107407], ['263108_page_23.png', 0.46450797], ['24547_page_66.png', 0.46306384], ['29864_page_4.png', 0.4387939], ['293281_page_253.png', 0.43617347], ['428455_page_49.png', 0.4296149], ['383482_page_4.png', 0.42750442], ['24577_page_142.png', 0.42371345], ['429093_page_40.png', 0.41194147], ['18971_page_107.png', 0.40585676], ['281748_page_5.png', 0.40515947], ['320363_page_291.png', 0.39642605], ['54136_page_56.png', 0.39614606], ['371085_page_36.png', 0.39350164], ['61784_page_40.png', 0.39287293], ['191419_page_23.png', 0.39197004], ['105117_page_27.png', 0.38968572], ['106106_page_41.png', 0.3888638], ['62251_page_32.png', 0.38885745], ['335945_page_113.png', 0.38813251], ['49157_page_1.png', 0.38804728], ['203699_page_15.png', 0.38735312], ['234233_page

Now we'll try the other way, modifying the original image to the shape of the new thing

In [12]:
db2 = {}

In [13]:
for i in range(len(png_paths)):
    imagePath = png_paths[i]
    image = util.im2tensor(util.load_image(imagePath))
    
    boi = ex_ref
    
    boi = boi.resize_(image.shape)
    
    #print(image.shape)
    
    distance = model.forward(boi, image)
    
    filename = imagePath[imagePath.rfind("/") + 1:]
    
    db2[filename] = distance

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
db2['107692_page_8'][0]

In [29]:
db['61784_page_40.png']

0.48306529568056483

In [30]:
lookupTable = []
similar_candidates = []
#score_names = []

In [31]:
for key in db.keys():
    if db[key] > 0.5:
        #lookupTable.append([key,db[key]])
        similar_candidates.append([key,db[key]])
        #score_names.append(db[key][0])

In [32]:
print(similar_candidates)

[['51212_page_385.png', 0.57772791977410498], ['88879_page_355.png', 0.50027010653499848], ['113413_page_1.png', 0.55664568208284038], ['109207_page_1.png', 0.54480797238550938], ['136341_page_51.png', 0.51216233267770839], ['146002_page_3.png', 0.53819045973017965], ['368980_page_1.png', 0.50095314671740332], ['54136_page_56.png', 0.53136800691571051], ['51265_page_52.png', 0.52285237613106261], ['51427_page_4.png', 0.51795235219421798], ['412274_page_386.png', 0.52641978771276066], ['88639_page_6.png', 0.50625401471355935], ['33060_page_107.png', 0.51675352287205789], ['136276_page_108.png', 0.52688949130002805], ['251051_page_103.png', 0.5019956240792085], ['19110_page_5.png', 0.50112751870762295], ['513452_page_139.png', 0.56739394916579444], ['412274_page_40.png', 0.58145918468852009], ['335127_page_13.png', 0.508645368605258], ['62185_page_34.png', 0.51798229275676555], ['53021_page_40.png', 0.52446429361859404], ['112888_page_187.png', 0.52588662251459506], ['06028_page_50.png',

In [35]:
similar_candidates = sorted(similar_candidates,key=lambda l:l[1], reverse=True)

In [36]:
print(similar_candidates)

[['412274_page_40.png', 0.58145918468852009], ['51212_page_385.png', 0.57772791977410498], ['29864_page_4.png', 0.57679942708445264], ['06028_page_50.png', 0.56764238060480576], ['513452_page_139.png', 0.56739394916579444], ['113413_page_1.png', 0.55664568208284038], ['109207_page_1.png', 0.54480797238550938], ['146002_page_3.png', 0.53819045973017965], ['54136_page_56.png', 0.53136800691571051], ['136276_page_108.png', 0.52688949130002805], ['412274_page_386.png', 0.52641978771276066], ['191419_page_23.png', 0.52599518371322185], ['112888_page_187.png', 0.52588662251459506], ['53021_page_40.png', 0.52446429361859404], ['51265_page_52.png', 0.52285237613106261], ['62185_page_34.png', 0.51798229275676555], ['51427_page_4.png', 0.51795235219421798], ['33060_page_107.png', 0.51675352287205789], ['136341_page_51.png', 0.51216233267770839], ['335127_page_13.png', 0.508645368605258], ['88639_page_6.png', 0.50625401471355935], ['251051_page_103.png', 0.5019956240792085], ['19110_page_5.png', 

In [37]:
top_7 = similar_candidates[0:6]

In [38]:
print(top_7)

[['412274_page_40.png', 0.58145918468852009], ['51212_page_385.png', 0.57772791977410498], ['29864_page_4.png', 0.57679942708445264], ['06028_page_50.png', 0.56764238060480576], ['513452_page_139.png', 0.56739394916579444], ['113413_page_1.png', 0.55664568208284038]]


Real match is 107692_page_8. Let's see what its score was.

In [40]:
db['107692_page_8.png']

0.40735924524979994

In [41]:
print(len(similar_candidates))

25
